## Verifying Recursive Formula 
#### (for Counting Unimodal Forests/ Parking Functions for a spider 1-graph)

### Necessary Functions:

In [97]:
def Bond_Lattice(g, n):
    ''' Creates a bond lattice for a given graph 
    params: g = a graph over n vertices
            n = the number of vertices
    return: A Bond Lattice for the set partitions of n elements'''
    
    partitions = SetPartitions(n).list() #find all set partitions 
    
    bad_partitions = [] #create lists to document partitions to include in the bond lattice or to leave out
    good_partitions = []
    
    for partition in partitions: #looking at each partition indvidually
        good_partition = True #automatically assume it creates a connected subgraph (innocent until proven guilty)

        for block in partition: #look at a block of the partition                
            new_graph = g.subgraph(vertices = block) #create an induced subgraph of the block
            good_partition = new_graph.is_connected() #ask whether the subgraph of the partition is connected
            if good_partition == False: #if its not connected...
                bad_partitions.append(partition) #add it to the list of bad partitions
                break #stop looking at this partition
            
        if good_partition: #if the induced subgraph of every block are all connected...
            good_partitions.append(partition) #add it to the list of good partitions
        
    return posets.SetPartitions(n).subposet(good_partitions) #return a poset of only the good partitions

def maximal_chains(BL):
    '''Gives the number of maximal chains of a bond lattice.
    Input: BL = the bond lattice
    Return: the number of maximal chains in the lattice'''
    return len(BL.maximal_chains())

def stanleys_bijection(BL):
    '''Computes Stanley's bijection on a noncrossing Bond Lattice
    input: a Bond Lattice of noncrossing partitions
    return: a list of all parking functions provided via Stanleys Bijection'''
    all_pfs = [] #create an empty list
    chains = BL.maximal_chains() #finds all maximal chains of the given bond lattice
    for chain in chains: #iterate through each chain
        parking_func = get_parking_function(chain) #retrieves the parking function for a given chain
        all_pfs.append(parking_func) #adds this parking function to the total list
    return all_pfs

def get_parking_function(chain):
    '''Computes the parking function for a given maximal chain
    input: maximal noncrossing chain of a poset
    return: the parking function of the given chain'''
    chain_length = len(chain) #number of elements in the chain
    parking_function = [] #create an empty parking function
    for index in range(0, chain_length - 1):
        x1 = 0
        x2 = 0
        for block in chain[index]: #identifying which blocks are combined in the next partition
            if block not in chain[index+1]: 
                if x1 == 0: 
                    x1 = block
                else:
                    x2 = block
        if min(x1) < min(x2):
            smaller = x1
            bigger = x2
        else:
            smaller = x2  
            bigger = x1
        parking_pref = 0
        for element in smaller:
            if element < min(bigger):
                if element > parking_pref:
                    parking_pref = element
        parking_function.append(parking_pref)
    return list(parking_function)

#### function to make cycle graphs:

In [98]:
def cycle_graphs(n):
    ''' Creates a cycle graph on n vertices
    input: n = number of vertices
    output: a cycle graph on n vertices'''
    cyc = Graph()
    for i in range(1, n):
        cyc.add_edge(i, i + 1)
    cyc.add_edge(1, n)
    return cyc

In [99]:
# cycle_graphs(5).plot()

#### function to add triangulation edges to cycle graphs:

In [100]:
def triangulate(vertex, graph):
    ''' Create a spider-vertex graph for a given vertex
    input: vertex = a vertex to create a spider graph from
           graph = the graph to triangulate
    output: a triangulation spider graph'''
    triangulated = graph
    for v in graph.vertices():
        if vertex != v:
            if graph.has_edge(vertex, v) == False:
                triangulated.add_edge(vertex, v)
    triangulated.set_pos(triangulated.layout_circular())
    return triangulated

In [101]:
# triangulate(1, cycle_graphs(5)).plot()

#### to get parking functions of a spider-1 k-gon:

In [102]:
def pf_spider1(n):
    triangulate(1, cycle_graphs(n)).plot()
    return maximal_chains(Bond_Lattice(triangulate(1, cycle_graphs(n)),n))

### The Recursive Formula:

In [103]:
# p(n) = n (p(n-1)) + sum_{k=3}^{n-1}n-2 Choose k - 2 p(k-1) p(n-k+1)
# p(n) = number of parking functions of the 1 - spider n-gon

In [104]:
def type1(n):
    '''Calculate the number of Type1 forests for a 1-spider graph on n vertices
    input: n = number of vertices on the triangulation graph
    output: the number of type 1 forests or parking functions with this characteristic'''
    return n*pf_spider1(n - 1)

In [105]:
def type2(n, k = 3, tot = 0):
    '''Calculate the total number of Type2 forests for a parking function of length n - 1 
    (a spider-1 graph on n vertices)
    input: n = length of parking function
           k = starting value in binomial coefficient
           tot = running total
    output: number of Type2 forests'''
    total = tot
    while k <= n - 1:
        total += binomial(n-2, k-2) * pf_spider1(k - 1) * pf_spider1(n - k + 1)
        return type2(n, k = k+1, tot = total)
    return total

In [106]:
def unimodal_forests_count(n):
    '''Computes the number of unimodal forests for a spider-1 graph on n vertices.
    Note: this is also the number of parking functions for a spider-1 graph on n vertices
    input: n = the number of vertices on the graph
    output: the total number of unimodal forests'''
    t1 = type1(n)
    t2 = type2(n)
    return t1 + t2

##### testing:

In [107]:
type1(5)

70

In [108]:
type2(5, k = 3)

18

In [109]:
unimodal_forests_count(5)

88

## + How does the number of type 1 or type 2 trees change when you increase the number of sides in a shape

In [110]:
for i in range(3, 10):
    print(unimodal_forests_count(i))

3
14
88
694
6578
72792
920904


In [112]:
for i in range(3, 10):
    print(type1(i), '/', type2(i), '/', unimodal_forests_count(i))

3 / 0 / 3
12 / 2 / 14
70 / 18 / 88
528 / 166 / 694
4858 / 1720 / 6578
52624 / 20168 / 72792
655128 / 265776 / 920904
